In [1]:
from model_loader import *
import torch
from transformers import pipeline
import gradio as gr
# dangerous to run but needed
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import pandas as pd

In [2]:
# testing base model
# pipe = pipeline(model="openai/whisper-tiny")  # change to
#
# def transcribe(audio):
#     text = pipe(audio)["text"]
#     return text
#
# iface = gr.Interface(
#     fn=transcribe,
#     inputs=gr.Audio(source="microphone"),
#     outputs="text",
#     title="Whisper Small Hindi",
#     description="Realtime demo for Hindi speech recognition using a fine-tuned Whisper small model.",
# )
#
# iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [ ]:
from scipy.io import wavfile
samplerate, audio = wavfile.read('000010001.WAV')
print(samplerate)
print(audio)
pipe = pipeline(model="openai/whisper-tiny")
print('pipedone')
text = pipe(audio)['text']
print(text)

In [ ]:
# testing lora model

lora_model = ws.from_pretrained("openai/whisper-small")
lora_model, params = lora_init(lora_model)
lora_model, params = lora_init(lora_model, modules=params)

pipe = pipeline(model = model)
iface = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(source="microphone", type="filepath"),
    outputs="text",
    title="Whisper Small Hindi",
    description="Realtime demo for Hindi speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()

competition code for fine tuning

In [4]:
from model_loader import *
import torch
from transformers import pipeline, WhisperFeatureExtractor, WhisperTokenizer
from datasets import load_dataset, Dataset, Audio
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from transformers import WhisperProcessor
import evaluate
import pandas as pd
from word2number import w2n

In [2]:
train = pd.read_csv("C:\\Users\\hojon\\OneDrive\\Desktop\\IRS Data\\TIL\\comps data\\Train.csv")
test = pd.read_csv("C:\\Users\\hojon\\OneDrive\\Desktop\\IRS Data\\TIL\\comps data\\Test_Novice_19May.csv")


def preprocess_data(df, local_path):
    """
    :param df:
    :param local_path: path to audio train files
    :return:
    """
    # converting annotations to lowercase
    df['annotation'] = [annotation.lower() for annotation in df['annotation']]

    # changing the numbers (letter form) to digits
    temp = []
    word2num = {'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9,
                'ten': 10, 'eleven': 11, 'twelve': 12, 'thirteen': 13, 'fourteen': 14, 'fifteen': 15, 'sixteen': 16,
                'seventeen': 17, 'eighteen': 18, 'nineteen': 19, 'twenty': 20, 'thirty': 30, 'forty': 40, 'fifty': 50,
                'sixty': 60, 'seventy': 70, 'eighty': 80, 'ninety': 90, 'zero': 0}
    for annotation in df['annotation']:
        sentence_added = False
        for word in word2num.keys():
            if word in annotation and sentence_added == False:
                index = annotation.index(word)
                new_annotation = annotation[:index] + str(word2num[word]) + annotation[index + len(word):]
                temp.append(new_annotation)
                sentence_added = True

        if sentence_added == False:
            temp.append(annotation)

    df['annotation'] = temp

    # making the first letter uppercase, add full stop to the back
    temp = []
    for annotation in df['annotation']:
        new_annotation = annotation[0].upper() + annotation[1:] + "."
        temp.append(new_annotation)
    df['annotation'] = temp

    # changing the filepaths
    temp = []
    for path in df['path']:
        new_path = local_path + path[6:]

        temp.append(new_path)
    df['path'] = temp

    audio_dataset = Dataset.from_dict({"audio": df['path'], "annotation": df['annotation']}).cast_column("audio", Audio())

    return audio_dataset


audio_dataset = preprocess_data(train, "C:\\Users\\hojon\\OneDrive\\Desktop\\IRS Data\\TIL\\comps data\\Train\\Train\\audio\\")

In [3]:
def preprocess_data(batch):
    feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")
    tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="English", task="transcribe")

    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]
    # compute log-Mel input features from input audio array
    input_features = []
    for instance in audio:
        input_features.append(feature_extractor(instance['array'], sampling_rate=16000).input_features[0])
    batch["input_features"] = input_features

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["annotation"]).input_ids

    return batch

processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="English", task="transcribe")
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

trainset = audio_dataset[:int(3750 * 0.8)]
testset = audio_dataset[int(3750 * 0.8):]


In [4]:
testset = preprocess_data(testset)
testset = Dataset.from_dict(testset)
trainset = preprocess_data(trainset)
trainset = Dataset.from_dict(trainset)
print(trainset)

Dataset({
    features: ['audio', 'annotation', 'input_features', 'labels'],
    num_rows: 750
})


In [5]:
testset = testset.remove_columns(['audio', 'annotation'])
trainset = trainset.remove_columns(['audio', 'annotation'])

In [6]:
len(testset[0]['input_features'][0])

3000

In [7]:
import evaluate

metric = evaluate.load("wer")

In [8]:
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="English", task="transcribe")
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [9]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

In [10]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [11]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-finetunev1",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=10,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=False,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=30,
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [12]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [13]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=trainset,
    eval_dataset=testset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

C:\Users\hojon\PycharmProjects\noobsbelow\ML\stt\huggingface-version\./whisper-small-finetunev1 is already a clone of https://huggingface.co/casual/whisper-small-finetunev1. Make sure you pull the latest changes with `repo.git_pull()`.


In [14]:
processor.save_pretrained(training_args.output_dir)

In [15]:
trainer.train()

C:\Users\hojon\anaconda3\envs\AI_env\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss



KeyboardInterrupt

